# Generating Random 0-1 Sheaves
This notebook defines `randomsheaf(n)`, which generates a random 0-1 sheaf on an $n$-simplex.

---

In [167]:
Plots.plot(betti2(randomsheaf(5)),title="Betti Numbers",xlims=(1,6),yticks = 0:9, ylims=(0,8))

---
## Packages and macros

Next, we call the necessary packages. `Iterators` lets us deal with subsets.

In [2]:
using Iterators

`issubset` takes two sets and checks whether the first is a subset of the second. We curry this function so we can apply it to a list

In [3]:
function issubsetcurry(x)
    newfunction = function (y) return issubset(x,y) end
    return newfunction
end

issubsetcurry (generic function with 1 method)

`extract1cofaces` gets the columns of the boundary operator that represent cofaces of the cell $A=x$, where $x$ is a subset of $\{1,\dots,dim+1\}$. 

In [4]:
function extract1cofaces(x,dim)
    return map(issubsetcurry(x),collect(subsets(collect(1:dim),length(x)+1)))
end

extract1cofaces (generic function with 1 method)

---
## Linear algebra over $F_2$

We copy in the necessary functions from `rref2`.

In [5]:
function swaprows(X, i, j)
     for k = 1:size(X,2)
        X[i,k],X[j,k] = X[j,k],X[i,k]
     end
     return X
end

swaprows (generic function with 1 method)

In [6]:
function rref2(M)
    col = 1                
    M = mod(M,2)
    for row = 1:size(M,1)
        #part 1: ensure pivot row has nonzero lead entry
        i = copy(row)
        while M[i,col] == 0
            i += 1   
            if i > size(M,1)
                col+=1
                if col > size(M,2)
                    return M
                end
                i = copy(row)
            end
        end        
        swaprows(M,i,row)
            
        #part 2: use pivot row to clear        
        for j = row+1:size(M,1)
            if M[j,col] != 0
                for k = 1:size(M,2)
                    M[j,k] = M[j,k]-M[row,k]
                end
            end
        end
        
        #part 3: reduce mod 2
        M = mod(M,2)
    end    
    return M
end

rref2 (generic function with 1 method)

In [7]:
function nullspace2(M)
    Mx = rref2(vcat(M,eye(Int64,size(M,2)))')    
    null = Array(Int64,size(M,2),0)

    
    for k = rank(rref2(M))+1:size(M,2)
        null = hcat(null,Mx[k,size(M,1)+1:size(M,2)+size(M,1)])
    end
    return null
end

nullspace2 (generic function with 1 method)

---
## Betti numbers

`betti2` takes a chain complex over $F_2$ and returns a list of its Betti numbers. Note `B[i]` is the $i-1$th Betti number. 

In [8]:
function betti2(D::Array{Array{Int64,2},1})
    B = Array(Int64,0)
    push!(B,size(nullspace2(D[1]),2)) # zeroth cohomology
    for i = 1:length(D)-1 
        push!(B,size(nullspace2(D[i+1]),2)+size(nullspace2(D[i]),2)-size(D[i],2))
    end
    push!(B,size(D[end],2)-size(nullspace2(D[end]),2)) # nth cohomology
    return B
end

betti2 (generic function with 1 method)

---
## Random Sheaves

This function generates a random 0-1 sheaf of dimension $n$, returning it as the chain complex of boundary operators.

To generate each row of a boundary operator, we compute the kernel of a restriction of the boundary operator in the dimension above to a suitable subspace of the domain. This involves knocking out the right columns. Here is an operator that, given a dimension $k$, a matrix $D_k$ of attaching maps from dimension $k$ to $k+1$, and a $k-1$-dimensional cell $A$, returns a basis for maps from $A$ to the $k$-dimensional cells that satisfy the required commutativity condition. This basis is a basis for the restriction of 

In [9]:
function randomsheaf(dim)
    D = Array(Array{Int64,2},0) 
    D_k = [0] 
    for k = dim:-1:1 # we construct the sheaf layer by layer, from the top dimension downwards
        D_kplus1 = copy(D_k) # takes a copy of the (k+1)th boundary operator
        D_k = Array(Int64,binomial(dim+1,k+1),0)
        
        for j in collect(subsets(collect(1:dim+1),k)) # j ranges over k-cells (these are subsets of dim+1 of size k+1)
            Aux = copy(D_kplus1[:,extract1cofaces(j,dim+1)]) # Aux takes the columns of the (k+1)th boundary operator corresponding to the cofaces of j  
            M = nullspace2(Aux) # find a basis for the kernel of restricted operator
            V_j = vec(M*rand(0:1,1,size(M,2))') # take a random element of the kernel -- we've now chosen our maps out of j
            for i in find(map(!,extract1cofaces(j,dim+1))) # turns our chosen maps into the jth column of the kth boundary operator
                insert!(V_j,i,0)
            end
            D_k = [D_k V_j] # adds this jth column in
        end
        unshift!(D,D_k) # inserts our new D_k into the vector of boundary operators
    end
    return D
end

randomsheaf (generic function with 1 method)